In [8]:
# plot ra một vài ví dụ ảnh mỗi class của dataset cifar-100
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [9]:
# chỉ plot ảnh ví dụ của dataset cifar-100 
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = "cifar100"
data_dir = "./data"
batch_size = 256 

from data.cifar100 import get_cifar100

train_loader, test_loader = get_dataloader(
    dataset_name=dataset,
    data_dir=data_dir,
    batch_size=batch_size,
    num_workers=4,
)

classes = train_loader.dataset.classes
class_to_idx = train_loader.dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')
    plt.show()

dataiter = iter(train_loader)
images, labels = dataiter.next()    
# lấy 100 class
num_classes_to_show = 100
fig, axes = plt.subplots(num_classes_to_show, 5, figsize=(10, 2 * num_classes_to_show))
for class_idx in range(num_classes_to_show):
    class_images = images[labels == class_idx]
    for i in range(5):
        ax = axes[class_idx, i]
        img = class_images[i]
        img = img / 2 + 0.5  # unnormalize
        npimg = img.numpy()
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        ax.axis('off')
        if i == 0:
            ax.set_title(idx_to_class[class_idx])
plt.tight_layout()
plt.show()

NameError: name 'get_dataloader' is not defined

In [ ]:
# quan sát sự khác nhau giữa các phương pháp contrastive learning khác nhau trên biểu đồ tsne